# SOTA INVESTIGATION FOR SENTIMENT ANALYSIS

## Load resources

In [ ]:
from emotion_classifier import *
import os

In [2]:
with open(os.path.join("secrets", "credentials.txt"), "r") as file:
    for line in file.readlines():
        key, value = line.split(":")

        if key == "huggingface":
            access_token = value

## Execute experiments

In [ ]:
available_datasets = list(DatasetHandler.DATASET_CONFIGS.keys())
print(f"The available datasets are: {available_datasets}")

In [ ]:
print("Available templates:")
PromptTemplate.TEMPLATES

### GO_EMOTIONS

In [4]:
dataset = DatasetHandler("go_emotions", "test")

In [ ]:
dataset.label2id

RoBERTa Fine-tuned with go_emotions

In [ ]:
# Transformers model evaluation
model = TransformersStrategy("SamLowe/roberta-base-go_emotions")
classifier = EmotionClassifier(dataset, model)

classifier.evaluate()
_ = classifier.create_performance_report(print_report=True)

Phi 3.5 mini quantized - ZERO SHOT

In [6]:
system_prompt = f"""
You are an assistant that classifies text into ALL applicable emotion labels from this list: {", ".join(dataset.labels)}. 
"""
user_pre_prompt = f"""
From this text, select the predominant emotions and respond with the final comma-separated predominant emotion names only:
"""

In [ ]:
# Llama.cpp model evaluation
prompt_template = PromptTemplate( system_prompt, user_pre_prompt )
model = LlamaCppStrategy(
    model_name = "Phi-3.5-mini-instruct-Q4_K_M-zeroshot",
    model_path = "models/Phi-3.5-mini-instruct-Q4_K_M.gguf", 
    n_ctx = 1024,
    prompt_template = prompt_template
)
classifier = EmotionClassifier(dataset, model)
classifier.evaluate(verbose=True)
report = classifier.create_performance_report(print_report=True)

Phi 3.5 mini quantized - FEW SHOT

In [9]:
user_pre_prompt = f"""Classify sentiment based on these examples:

Text: [NAME], loved the story. Personally I don’t think this was petty I think it was quite fair and right. Good job dude.
Emotions: admiration, love

Text: Oh that's right they just came back from the vacation. This is a vacation business trip. What a fuckin' joke.
Emotions: amusement

Text: Players like this makes me rage quit tbh.
Emotions: anger, annoyance

Text: If you had a giant rock on your land, and you couldn't stop people climbing, wouldn't you be pissed?
Emotions: annoyance, confusion, curiosity

Text: This very thing got me diagnosed as ADD. I'm of the drugs now and feeling way much better. 
Emotions: approval, relief

Text: Get rid. This is a toxic relationship that is making you unhappy. You are making all efforts for nothing in return.
Emotions: caring, realization

Text: Isn't it the same thing? Or at least equally bad?
Emotions: confusion

Text: It's great that you're a recovering addict, that's cool. Have you ever tried DMT?
Emotions: curiosity, admiration

Text: I know. And it's very selfish of me to say, but I just wish it was different people that cared.
Emotions: desire

Text: It's really very upsetting to have parents who dismiss this condition.
Emotions: disappointment

Text: We got nowhere with that because you only drop one-liners and refuse to actually engage.
Emotions: disapproval

Text: Apparently, he can't just have sex with anyone because he needed to rape someone.
Emotions: disgust

Text: I was teased for being a virgin when I was a 6th grader- in 2005
Emotions: embarrassment

Text: I’d love to get an update on what happened! (When you are ready)
Emotions: excitement, love

Text: I've also heard that intriguing but also kinda scary
Emotions: fear

Text: I didn't know that, thank you for teaching me something today!
Emotions: gratitude

Text: [NAME] death is just so..... senseless. Why? WHY??? The based gods have forsaken us
Emotions: grief

Text: A surprise turn of events! I'm so glad you heard such great things about yourself!
Emotions: joy, surprise

Text: I need to just eliminate anxiety and not have cravings
Emotions: nervousness

Text: I'm going to hold out hope for something minor even though it looked really bad. Just going to wait for the official news.
Emotions: optimism

Text: Of course I love myself because I'm awesome.
Emotions: pride

Text: Oh yeah, I forgot about the mean one.
Emotions: realization

Text: Resetting a dislocated knee hurts like hell but it feels a lot better immediately after.
Emotions: relief

Text: I sincerely apologize...I’ll delete and please take this upvote
Emotions: remorse

Text: It's hard to make friends. :( I sit alone.
Emotions: sadness

Text: I can’t believe that’s real
Emotions: surprise

Text: I'd say it's pretty uncommon now.
Emotions: neutral

Text: 
"""

In [ ]:
# Llama.cpp model evaluation
prompt_template = PromptTemplate( system_prompt, user_pre_prompt )
model = LlamaCppStrategy(
    model_name = "Phi-3.5-mini-instruct-Q4_K_M-few_shot",
    model_path = "models/Phi-3.5-mini-instruct-Q4_K_M.gguf", 
    prompt_template = prompt_template
)
classifier = EmotionClassifier(dataset, model)
classifier.evaluate(verbose=True)
report = classifier.create_performance_report(print_report=True)

DeepSeek R1 Distill Qwen 1.5B quantized zero-shot

In [11]:
# As indicated in the DeepSeek R1 Distill Qwen 1.5B model, we shouldn't add a system prompt because it was trained without it, it'd only add noise
system_prompt = ""
user_pre_prompt = f"""
You are an assistant that classifies text into ALL applicable emotions from: {", ".join(dataset.labels)}. 
From this text, select the predominant emotions and respond with the final comma-separated predominant emotion names only:
"""

In [ ]:
prompt_template = PromptTemplate( system_prompt, user_pre_prompt )
model = LlamaCppStrategy(
    model_name = "DeepSeek-R1-Distill-Qwen-1.5B-Q8_0-zero_shot",
    model_path = "models/DeepSeek-R1-Distill-Qwen-1.5B-Q8_0.gguf",
    prompt_template = prompt_template
)

Gemma 2 2B quantized zero-shot

In [12]:
system_prompt = f"""
You are an assistant that classifies text into ALL applicable emotions from: {", ".join(dataset.labels)}. 
"""
user_pre_prompt = f"""
From this text, select the predominant emotions and respond with the final comma-separated predominant emotion names only:
"""

In [ ]:
prompt_template = PromptTemplate( system_prompt, user_pre_prompt )
model = LlamaCppStrategy(
    model_name = "gemma2-2b-zero_shot",
    model_path = "models/dolphin-2.9.4-gemma2-2b-Q6_K_L.gguf",
    prompt_template = prompt_template
)
classifier = EmotionClassifier(dataset, model)
classifier.evaluate(verbose=True)
report = classifier.create_performance_report(print_report=True)

## Print results

In [ ]:
model_report = model.create_performance_report(print_report=True)

## Inference and model testing

In [ ]:
sentiment_template = f"Respond to this message only replying with the single emotion label you can detect from these emotion labels: {model.dataset_labels} for the following piece of text: "
model.run_inference(sentiment_template + "My mom said she wants me to go and never return back again")